# TF-IDF

# GLoVe X TFIDF

In [1]:
import pandas as pd

psalms = pd.read_csv("data/psalms_with_vectors.txt")

In [ ]:
def compute_query_embedding(query):
    tokens = query.lower().split()  # lowercase, remove punctuation, tokenize
    
    numerator = np.zeros(embedding_dim)
    denominator = 0

    for word in tokens:
        if word in glove_vectors and word in idf_weights:
            weight = idf_weights[word]  # or full TF-IDF if TF available
            numerator += glove_vectors[word] * weight
            denominator += weight

    if denominator == 0:
        return None

    return numerator / denominator


In [2]:
def search_glove_tfidf(query):
    q_vec = compute_query_embedding(query)
    results = []

    for doc_id, vec in doc_embeddings.items():
        sim = np.dot(q_vec, vec) / (np.linalg.norm(q_vec) * np.linalg.norm(vec))
        results.append((doc_id, sim))

    # Sort by similarity
    results = sorted(results, key=lambda x: x[1], reverse=True)

    # Top 5 results
    for doc_id, sim in results[:5]:
        print(f"{doc_id}: {sim:.4f}")